In [38]:
# Author: Davide Aloi
# Description: Opens each joystick session and verifies if there are voxels that survive 
# thresholding (p=0.001 --> t = 3.13)

import nilearn
from nilearn import image
import glob
import os
import numpy as np
from nilearn import reporting

path_rois = 'C:\\Users\\davide\\Documents\\GitHub\\raindrop_analyses_fmri_eeg\\fmri\\rois\\'
m1 = image.load_img(glob.glob(path_rois + 'wm1.nii')[0]) # m1 roi in the native space
sma = image.load_img(glob.glob(path_rois + 'wsma.nii')[0]) # sma roi in the native space
th = image.load_img(glob.glob(path_rois + 'wth.nii')[0]) # th roi in the native space
rois = [m1, sma, th]
roi_name = ['m1', 'sma', 'th']

threshold = 3.13 # t threshold for p 0.001 uncorrected

In [33]:
# creating list with SPM_t results for the joystick task
path_res_p0 = 'D:\\Raindrop_data\\p01\\'
path_joystick_results = glob.glob(path_res_p0 + 'p01_*\\day*\\fmri_data\\JOYSTICK*\\nifti\\glm_first_level\\spmT_0002.nii')

In [73]:
# looping through spm_t maps applying the 3 rois to each spm_t
# then checking if suprathreshold voxels are present
for spm_t in path_joystick_results:
    print('Opening ' + spm_t)
    spm_t_img = image.load_img(spm_t)

    for n_roi, roi in enumerate(rois):

        print('Applying ROI: ' + roi_name[n_roi])
        roi_resampled = image.resample_to_img(roi, spm_t_img, interpolation = 'nearest')
        spm_t_img_masked = np.where(roi_resampled.get_fdata() == 1, spm_t_img.get_fdata(), 0)
        # counting suprathreshold voxels
        
        supr_voxels = (spm_t_img_masked.flatten() > threshold).sum()
        print('number of significant voxels: ' + str(supr_voxels))

        if supr_voxels > 0:
            table = reporting.get_clusters_table(image.new_img_like(spm_t_img, spm_t_img_masked), threshold, 1, min_distance = 8)
            table['X'] = table['X'].round().astype(int)
            table['Y'] = table['Y'].round().astype(int)
            table['Z'] = table['Z'].round().astype(int)

            print(table.round(3))

# Results can be verified by loading the spm.mat file and applying a ROI
# The only significant results are in week 6 day 1 joystick baseline (ROIs: m1 and sma)
# NB the tables resulting from get_clusters_table might differ slighly from SPM ones
# as SPM outputs the first 3 local maxima only.

Opening D:\Raindrop_data\p01\p01_w02\day01\fmri_data\JOYSTICK_BASELINE_0015\nifti\glm_first_level\spmT_0002.nii
Applying ROI: m1
number of significant voxels: 0
Applying ROI: sma
number of significant voxels: 0
Applying ROI: th
number of significant voxels: 0
Opening D:\Raindrop_data\p01\p01_w02\day01\fmri_data\JOYSTICK_POST_0019\nifti\glm_first_level\spmT_0002.nii
Applying ROI: m1
number of significant voxels: 0
Applying ROI: sma
number of significant voxels: 0
Applying ROI: th
number of significant voxels: 0
Opening D:\Raindrop_data\p01\p01_w02\day05\fmri_data\JOYSTICK_POST_0010\nifti\glm_first_level\spmT_0002.nii
Applying ROI: m1
number of significant voxels: 0
Applying ROI: sma
number of significant voxels: 0
Applying ROI: th
number of significant voxels: 0
Opening D:\Raindrop_data\p01\p01_w04\day01\fmri_data\JOYSTICK_BASELINE_0005\nifti\glm_first_level\spmT_0002.nii
Applying ROI: m1
number of significant voxels: 0
Applying ROI: sma
number of significant voxels: 0
Applying ROI: th


0   -10
1     0
Name: X, dtype: int32

In [61]:
2.5*2.5*2.5

15.625